<a href="https://colab.research.google.com/github/PavanKumarDharmoju/weights-and-biases/blob/main/Weights_and_biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Weights abd biases

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [ ]:
!pip install wandb -qU

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [ ]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [ ]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [ ]:
diamonds.shape

(53940, 10)

In [ ]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [ ]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:53:41] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:53:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [ ]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [ ]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:53:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [ ]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:53:43] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[102]	train-rmse:376.20429	validation-rmse:532.59813


In [ ]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:53:44] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [ ]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [ ]:
# Log in to your W&B account
import wandb

wandb.login()

wandb: Currently logged in as: pavandharmoju (pavanresearch). Use `wandb login --relogin` to force relogin


True

In [ ]:
sweep_config = {
    'method': 'random',  # Can be 'grid', 'random', 'bayesian'
    'metric': {
        'name': 'rmse',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.01,
            'max': 0.2
        },
        'max_depth': {
            'values': [3, 5, 7, 9]
        },
        'subsample': {
            'min': 0.6,
            'max': 0.9
        },
        'colsample_bytree': {
            'min': 0.6,
            'max': 0.9
        },
        'n_estimators': {
            'values': [50, 100, 150, 200]
        }
    }
}


In [ ]:
import wandb

sweep_id = wandb.sweep(sweep_config, project="xgboost_diamonds_sweep_experiments", entity='pavandharmoju')


Create sweep with ID: nrzboqad
Sweep URL: https://wandb.ai/pavandharmoju/xgboost_diamonds_sweep_experiments/sweeps/nrzboqad


In [ ]:
def train():
    # Initialize a W&B run
    run = wandb.init()

    # Access the hyperparameters through wandb.config
    config = wandb.config

    # Define the model
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': config.learning_rate,
        'max_depth': int(config.max_depth),
        'subsample': config.subsample,
        'colsample_bytree': config.colsample_bytree,
        'n_estimators': int(config.n_estimators),
        'eval_metric': 'rmse'
    }

    # Train the model
    model = xgb.train(params, dtrain, num_boost_round=config.n_estimators)

    # Evaluate the model
    predictions = model.predict(dtest)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    # Log metrics
    wandb.log({'rmse': rmse})

    run.finish()


In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 6gfns4qy with config:
wandb: 	colsample_bytree: 0.8721852812160612
wandb: 	learning_rate: 0.10543401200754732
wandb: 	max_depth: 5
wandb: 	n_estimators: 200
wandb: 	subsample: 0.8827807666925989
wandb: Currently logged in as: pavandharmoju. Use `wandb login --relogin` to force relogin


rmse,▁
rmse,533.9805


wandb: Agent Starting Run: wp0kn5ky with config:
wandb: 	colsample_bytree: 0.7233831099982967
wandb: 	learning_rate: 0.09199680902788608
wandb: 	max_depth: 3
wandb: 	n_estimators: 200
wandb: 	subsample: 0.7715684068652043


rmse,▁
rmse,576.59301


wandb: Agent Starting Run: 1q7y3qv5 with config:
wandb: 	colsample_bytree: 0.7050223344036699
wandb: 	learning_rate: 0.062081427442106735
wandb: 	max_depth: 9
wandb: 	n_estimators: 50
wandb: 	subsample: 0.7500189687798899


...

wandb: Agent Starting Run: rp93klp7 with config:
wandb: 	colsample_bytree: 0.7983299733272369
wandb: 	learning_rate: 0.06151642810780787
wandb: 	max_depth: 3
wandb: 	n_estimators: 200
wandb: 	subsample: 0.7232341550797495


rmse,▁
rmse,600.45896


wandb: Agent Starting Run: j7hwjuyh with config:
wandb: 	colsample_bytree: 0.7087480918840534
wandb: 	learning_rate: 0.08908882135167041
wandb: 	max_depth: 7
wandb: 	n_estimators: 150
wandb: 	subsample: 0.6387753392249019


rmse,▁
rmse,535.64313


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Hyperparameters and RMSE Observations

### Hyperparameter Variability and Impact:

- **colsample_bytree**: This parameter varied across runs from about 0.639 to 0.892. Changes in this parameter, which controls the fraction of features (by number) that will be used to train each tree, seem to influence the RMSE. However, a clear trend is not directly observable from the logs alone, suggesting the interplay with other parameters affects its impact.
  
- **learning_rate**: Varied between about 0.017 and 0.16. Generally, models with mid-range learning rates (around 0.1) tended to perform better, indicating too slow or too fast learning rates might be suboptimal.
  
- **max_depth**: Values ranged from 3 to 9. The depth of trees can significantly influence overfitting; shallower trees generally lead to higher biases while deeper trees might overfit, particularly in smaller datasets.
  
- **n_estimators**: This had values like 50, 100, 150, and 200. More trees generally improve model performance but up to a point, after which improvements might plateau or even decline due to overfitting.
  
- **subsample**: Ranged from about 0.638 to 0.882. This parameter sets the fraction of the training data to be randomly sampled for each tree. Higher values generally provide a better chance at model robustness by reducing variance.

### Performance Analysis:

- The RMSE scores ranged significantly, from a low of about 533 to a high of 1868. Lower RMSE scores indicate better model predictions. The run with the highest RMSE (1868) used particularly low `learning_rate` (0.017), relatively high `max_depth` (7), and a low number of `n_estimators` (50), possibly indicating insufficient learning phase.
  
- The best performing models (lowest RMSEs around 533) had more balanced configurations, avoiding extremes in learning rates and using a moderate number of trees.
